## Potential Energy Surface (PES)

The potential energy surface (PES) of a molecular system describes its energy in terms of geometric parameters such as internuclear distances. For diatomic molecules with only one degree of freedom, i.e. one bond length, the surface is reduced to a potential energy curve or Morse potential.

![image](https://user-images.githubusercontent.com/39799035/155658637-f9b76e51-ef90-4afa-a46b-fb2cda23efd4.png)

Knowledge of the PES allows the exploration of molecular properties, such as atomization or binding energy, and equilibrium bond lengths for various molecules. For more complex molecules, the optimal equilibrium bond lengths and angles for molecules can be studied. Also, the PES along a particular path can be used to map the energy changes in chemical reactions, allowing study of reaction energies and activation barriers.

### Molecular Hamiltonian

The energy of the molecular system is the eigenvalues $E$ of Molecular Hamiltonian. $\hat{H}$ the represents the sum of the Kinetic and Potential energy operator of the whole molecule which consists of the electrons and nucleus. Finding the eigenvalues $E$ requires one to solve the time-independent Schrödinger Equation: 

$$\hat{H}|\phi_E\rangle=E|\phi_E\rangle$$

Without getting into too much detail about the chemistry, the Molecular Hamiltonian is usually expressed as a tensor (a generalised matrix) in terms of $m$ eigenstates of the Molecule that has no effective electronic interactions, known as molecular orbitals. This tensor is then futher decomposed into a linear decomposition of Pauli Strings, $\sigma_1 \otimes \ldots \otimes \sigma_m$, where $\sigma \in \{I,X,Y,Z\}$.

$$\hat{H} = \sum_{i}c_i\sigma_i + \sum_{ij}c_{ij}\sigma_i\sigma_j + \sum_{ijk}c_{ijk}\sigma_i\sigma_j\sigma_k + \ldots$$

Fortunately, for us, PennyLane has provided a neat function that quickly generates this decomposition for us to use: [`qml.qchem.molecular_hamiltonian()`](https://pennylane.ai/qml/demos/tutorial_quantum_chemistry.html).

Throughout this notebook, we shall assume the Jordan-Wigner electron occupation-to-qubit mapping, where the occupation of molecular orbital by the electron is represented as $|1\rangle$ while the unoccupation is represented as $|0\rangle$.

## Problem Inspired Ansatz 

### What is Disentangled UCC and Problem Inspired Ansatz?

Disentangled UCC [[1]](http://arxiv.org/abs/1910.10130) is a problem-inspired ansatz that is constructed based on established Quantum Chemistry methods. It is expected that a problem-inspired ansatz will naturally produce states close to the actual ground state solution, resulting in a good agreement of the ground state energy with the true solution. This ansatz uses Givens rotations circuit blocks which has a unique particle conservation property. It simply means that the ansatz will avoid quantum states that correspond to the wrong number of electrons. Givens rotations are highly suitable for Quantum Chemistry applications where most chemical problems do not involve changing total number of electrons. 

As a result, it is effective in narrowing the search in the expansive Hilbert space for a potential solution. Unfortunately, problem-inspired ansatze are much more difficult to implement on a real quantum hardware as they usually require deep quantum circuits which increase the unwanted effect of quantum hardware noise. This in turn greatly affects solution accuracies.


### How do I generate Disentangled UCC Ansatz?

Suppose a molecule with $n$ electrons and $m$ orbitals, then its Hartree-Fock Ground State is $|\psi_0\rangle=|1_1\ldots1_n0_{n-1}\ldots0_{m}\rangle$ in the computational basis of $m$ number of qubits, where index $1\le i,j\le n$ represents the occupied orbitals and index $n-1\le k,l\le m$ represents the unoccupied orbitals, such that $i<j<k<l$. The Hartree-Fock Ground State, $|\psi_0\rangle$ represents the ground state of a molecule without any effective electron interaction. To generate the Disentangled Unitary Coupled Cluster (UCC) Ansatz $|\psi_{PI}\rangle$, we apply a particular sequence of Singles $U_{i,k}(\theta_{i,k})$ and Double $U_{ij,kl}(\theta_{ij,kl})$ Excitation Rotations on the Hartree-Fock State $|\psi_0\rangle$ That is,
$$|\psi_{PI}\rangle = \prod_{\alpha}U_{\alpha}(\theta_\alpha)|1_1\ldots1_n0_{n-1}\ldots0_{m}\rangle$$
$$U_{\alpha}(\theta_\alpha) \in \{U_{i,k}(\theta_{i,k}),  U_{ij,kl}(\theta_{ij,kl})\}$$
where the index $\alpha$ iterates over a particular sequence of Single and Double Excitation Rotations. 

Note that in general, the acutal Disentangled UCC includes higher number of excitation such as Triples and Quadruples Excitations etc. For demonstration purposes, we shall only have Singles and Double Excitations in our Disentangled UCC Ansatz. The following steps are a modification of [[1]](http://arxiv.org/abs/1910.10130) that generates the sequence of $U_{\alpha}(\theta_\alpha)$ without redundant rotations:

1. Start with the highest occupied orbital index $p$.

2. Apply all Single Excitations $U_{i,k}(\theta_{i,k})$ where $i=p$, in the order of decreasing "excitation jumps" $(=k-i)$. 

3. Apply all Double Excitations $U_{ij,kl}(\theta_{ij,kl})$ where $i,j=p$, in the order of decreasing  "excitation jumps"$(=[k+l]-[i+j])$.

4. Go to step 1 and repeat with the next highest occupied orbital $p-1$ without re-using any previously chosen any single or double excitation.


### What are Singles and Doubles Excitations Rotations?

Singles and Doubles Excitations Rotations represents a process where correspondingly, one and two electrons are "excited" from the occupied orbitals to unoccpied orbitals. Explicitly in quantum computing language, the Single Excitation Rotation $U_{i,k}(\theta_{i,k})$ is rotation in a 2D subspace of $\{ |1_{i}0_{k}\rangle,|0_{i}1_{k}\rangle \}$ for any two chosen $i,k$ qubits and leaves the rest of computational basis unchanged. This rotation can be chemically intrepreted as a single electron being "excited" from the lower energy orbital $i$ to the higher energy orbital $k$ or "de-excited" if vice versa.
$$|0_{i}1_{k}\rangle \xrightarrow{U_{i,k}(\theta_{i,k})}\cos(\theta_{i,k}/2)|1_{i}0_{k}\rangle + \sin(\theta_{i,k}/2)|0_{i}1_{k}\rangle $$
$$|1_{i}0_{k}\rangle \xrightarrow{U_{i,k}(\theta_{i,k})}\cos(\theta_{i,k}/2)|1_{i}0_{k}\rangle - \sin(\theta_{i,k}/2)|0_{i}1_{k}\rangle $$

Double Excitation Rotation $U_{ij,kl}(\theta_{ij,kl})$ is rotation in a 2D subspace of $\{ |1_{i}1_{j}0_{k}0_{l}\rangle,|0_{i}0_{j}1_{k}1_{l}\rangle \}$ for any four chosen  $i,j,k,l$ qubits and leaves the rest of computational basis unchanged. This rotation can be intrepreted as a two electrons being "excited" from lower energy orbitals $i,j$ to the higher energy orbitals $k,l$ or "de-excited" if vice versa.
$$|0_{i}0_{j}1_{k}1_{l}\rangle \xrightarrow{U_{ij,kl}(\theta_{ij,kl})}\cos(\theta_{ij,kl}/2)|1_{i}1_{j}0_{k}0_{l}\rangle + \sin(\theta_{ij,kl}/2)|0_{i}0_{j}1_{k}1_{l}\rangle $$
$$|1_{i}1_{j}0_{k}0_{l}\rangle \xrightarrow{U_{ij,kl}(\theta_{ij,kl})}\cos(\theta_{ij,kl}/2)|1_{i}1_{j}0_{k}0_{l}\rangle - \sin(\theta_{ij,kl}/2)|0_{i}0_{j}1_{k}1_{l}\rangle $$


### What does Disentangled UCC Ansatz Acheive?

Effectively, the Disentangled UCC Ansatz state will become a linear combination of the Hartree-Fock Ground molecular state $|\psi_0\rangle$ and all other possible singly and doubly excited Hartree-Fock molecular basis state. Let us illustrate this using the simplest case of Hydrogen $H_2$ molecule with 2 electrons and 4 orbitals. The $H_2$ Hartree-Fock ground state is $|\psi_0\rangle=|1100\rangle$ with 4 qubits. Disentangled UCC ansatz will becomes a linear combination of the following computational basis $\{|1100\rangle,|0110\rangle,|0101\rangle,|1010\rangle,|1001\rangle,|0011\rangle\}$ where each the basis represents a possible electronic configuration of hydrogen molecule.

Note that other quantum basis such as $|1110\rangle$ or $|0010\rangle$ must be excluded from the ground state as it does not have the correct occupation number. To generate UCCSD ansatz for the Hydrogen Molecule, we carefully apply each unique Singles and Doubles Excitation Rotation once on the Hartree-Fock State $|1100\rangle$ such that all states are being generated without any redundant rotations.

In the our example, our hydrogen molecular ansatz will be generated by 5 disentangled excitations of the following:

$$\prod_{\alpha}U_{\alpha}=U_{1,3}U_{1,2}U_{01,23}U_{0,3}U_{0,2}$$

with the $\theta$  removed for clarity. We have provided a custom coded function that generate such disentangled single and double exitation sequence `qanything.get_index.generate_disentanglement_order()`.



### Molecules 

We shall give a simple demonstration of Hydrogen Molecule, Helium Dimer and Lithium Hydride.

#### Hydrogen Molecule (STO-3G)

Hydrogen Molecule is the everyone's favourite benchmark molecule as it is the simplest molecule with only two protons and two electrons that has no analytical solution to its energy and molecular orbitals. Using the minimal basis STO-3G, we can represent the ansatz with only 4 qubits. We shall build and optimise the quantum circuit ansatz on PennyLane before running it on various IBM Noise Model backends to quickly test if today's quantum devices are ready for quantum chemistry applications.


#### Helium Dimer (6-31G)

According to the periodic table, Helium is an noble gas, and by basic chemistry knowledge, it would mean that Helium would not be able to form covalent bonds like hydrogen molecule. However, Helium bond do exist [[2]](https://link.aps.org/doi/10.1103/PhysRevLett.85.2284) which are held by van der Waals forces with a bond length of about 98 Bohr. Despite this, unbeknownst to many, there exist a tiny minimum molecular energy at around 6 Bohr using the 6-31G atomic basis. We shall find out if such a minimum can be observed using our problem inspired ansatz.


#### Lithium Hydride (STO-3G)

Lithium Hydride is the simplest metal hydride with 4 electrons which can be produced by reacting lithium metal with hydrogen gas. This molecule poses a moderate implementation challenge due to need for finding a good classical optimiser that could quickly handle hundreds of variational parameter in the problem-inspired ansatz.


#### References

[[1]](http://arxiv.org/abs/1910.10130) Evangelista, F. A., Chan, G. K.-L., & Scuseria, G. E. (2019). Exact parameterization of fermionic wave functions via unitary coupled cluster theory. The Journal of Chemical Physics, 151(24), 244112.

[[2]](https://link.aps.org/doi/10.1103/PhysRevLett.85.2284) R. E. Grisenti, W. Schöllkopf, J. P. Toennies, G. C. Hegerfeldt, T. Köhler, and M. Stoll, (2000). Determination of the Bond Length and Binding Energy of the Helium Dimer by Diffraction from a Transmission Grating, Phys. Rev. Lett. 85, 2284.

In [2]:
# Native Lib
import itertools as it
import importlib
# External Lib
import pennylane as qml
from pennylane import numpy as plnp
import numpy as np
import matplotlib.pyplot as plt

# qanything Lib
import qanything.get_index as getind
import qanything.get_molecule as getmol

### Info: Hydrogen Molecule

In [5]:
# Create Hydrogen Molecule
mol_name = 'h2'
mol_full_name = 'hydrogen_molecule'
bond_length = 1.4
mol_symbols, coords, charge, num_elec = getmol.get_molecule(mol_name, bond_length)
print(f'Molecule Symbol: {mol_symbols}')
print(f'Coordinates: {coords}')
print(f'Molecule Charge: {charge}')
print(f"Number of Electrons: {num_elec}")

# Get the Molecular Hamiltonian (Using Jordan-Wigner)
mol_ham, num_qubits = qml.qchem.molecular_hamiltonian(symbols=mol_symbols, coordinates=coords, name=mol_full_name ,charge=charge, basis='sto-3g')
print(f'Number of Qubits: {num_qubits}')
print(f"Number of Pauli Strings: {len(mol_ham.terms[1])}")

# Getting the Orbital and Excitation Index 
occupied_index, single_index, double_index = getind.generate_excite_index(num_qubits, num_elec)


# Get the Fermionic Disentangled Unitary Coupled Cluster Single Doubles (UCCSD) Excitation Order 
disentangle_order = getind.generate_disentanglement_order(occupied_index, single_index, double_index)
# print(f"Disentangle UCCSD Excitation Order:\n{disentangle_order}")
print(f"Number of Single Excitations: {len(single_index)}")
print(f"Number of Double Excitations: {len(double_index)}")
print(f"Total Number of Excitations: {len(disentangle_order)}")

# Get the Hartree-Fock State
hf = qml.qchem.hf_state(electrons=num_elec, orbitals=num_qubits)
print(f"Hartree-Fock State: {hf}")

print(f"Occupied Index: {occupied_index}")
print(f"Single Excitation Index: \n{single_index}")
print(f"Double Excitation Index:\n{double_index}")

# Prepare the Device
dev = qml.device('default.qubit', wires=num_qubits)

@qml.qnode(dev) # Prepare the UCCSD Ansatz based on the Disentangled Order
def uccsd_ansatz(params, wires, excitations):
    qml.BasisState(hf, wires=wires)
    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        else:
            qml.SingleExcitation(params[i], wires=excitation)

    return qml.expval(qml.PauliZ(0))

print(f"Disentangle UCCSD Excitation Order:\n{disentangle_order}\n")

print(f'Molecular Hamiltonian:\n{mol_ham}')

# Draw the quantum circuit
drawer = qml.draw(uccsd_ansatz, max_length=200)
print(drawer(params=plnp.zeros(len(disentangle_order)), wires=range(num_qubits), excitations=disentangle_order))

Molecule Symbol: ['H', 'H']
Coordinates: [0.  0.  0.  0.  0.  1.4]
Molecule Charge: 0
Number of Electrons: 2
Number of Qubits: 4
Number of Pauli Strings: 15
Number of Single Excitations: 4
Number of Double Excitations: 1
Total Number of Excitations: 5
Hartree-Fock State: [1 1 0 0]
Occupied Index: [0 1]
Single Excitation Index: 
[[0 3]
 [0 2]
 [1 3]
 [1 2]]
Double Excitation Index:
[[0 1 2 3]]
Disentangle UCCSD Excitation Order:
[[1, 3], [1, 2], [0, 1, 2, 3], [0, 3], [0, 2]]

Molecular Hamiltonian:
  (-0.2230402039058882) [Z2]
+ (-0.2230402039058882) [Z3]
+ (-0.09815625531892702) [I0]
+ (0.1712824929287252) [Z0]
+ (0.17128249292872522) [Z1]
+ (0.12057651910822698) [Z0 Z2]
+ (0.12057651910822698) [Z1 Z3]
+ (0.16589099780617111) [Z0 Z3]
+ (0.16589099780617111) [Z1 Z2]
+ (0.1686485210819778) [Z0 Z1]
+ (0.17437383667114004) [Z2 Z3]
+ (-0.045314478697944145) [Y0 Y1 X2 X3]
+ (-0.045314478697944145) [X0 X1 Y2 Y3]
+ (0.045314478697944145) [Y0 X1 X2 Y3]
+ (0.045314478697944145) [X0 Y1 Y2 X3]
 0:

### Info: Helium Dimer (6-31G)

In [12]:
# Create Molecule
mol_name = 'he2'
mol_full_name = 'helium_dimer'
bond_length = 1.4
mol_symbols, coords, charge, num_elec = getmol.get_molecule(mol_name, bond_length)
print(f'Molecule Symbol: {mol_symbols}')
print(f'Coordinates: {coords}')
print(f'Molecule Charge: {charge}')
print(f"Number of Electrons: {num_elec}")

# Get the Molecular Hamiltonian (Using Jordan-Wigner)
mol_ham, num_qubits = qml.qchem.molecular_hamiltonian(symbols=mol_symbols, coordinates=coords, name=mol_full_name ,charge=charge, basis='6-31G')
# print(f'Molecular Hamiltonian:\n{mol_ham}')
print(f'Number of Qubits: {num_qubits}')
print(f"Number of Pauli Strings: {len(mol_ham.terms[1])}")

# Getting the Orbital and Excitation Index 
occupied_index, single_index, double_index = getind.generate_excite_index(num_qubits, num_elec)
print(f"Occupied Index: {occupied_index}")
# print(f"Single Excitation Index: \n{single_index}")
# print(f"Double Excitation  Index:\n{double_index}")

# Get the Fermionic Disentangled Unitary Coupled Cluster Single Doubles (UCCSD) Excitation Order 
disentangle_order = getind.generate_disentanglement_order(occupied_index, single_index, double_index)
# print(f"Disentangle UCCSD Excitation Order:\n{disentangle_order}") 
print(f"Number of Single Excitations: {len(single_index)}")
print(f"Number of Double Excitations: {len(double_index)}")
print(f"Total Number of Excitations: {len(disentangle_order)}")

# Get the Hartree-Fock State
hf = qml.qchem.hf_state(electrons=num_elec, orbitals=num_qubits)
print(f"Hartree-Fock State: {hf}\n")

# Prepare the Device
dev = qml.device('default.qubit', wires=num_qubits)

@qml.qnode(dev) # Prepare the UCCSD Ansatz based on the Disentangled Order
def uccsd_ansatz(params, wires, excitations):
    qml.BasisState(hf, wires=wires)
    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        else:
            qml.SingleExcitation(params[i], wires=excitation)

    return qml.expval(qml.PauliZ(0))

print(f"Disentangle UCCSD Excitation Order:\n{disentangle_order}\n") 

# Draw the quantum circuit
drawer = qml.draw(uccsd_ansatz, max_length=100)
print(drawer(params=plnp.zeros(len(disentangle_order)), wires=range(num_qubits), excitations=disentangle_order))

Molecule Symbol: ['He', 'He']
Coordinates: [0.  0.  0.  0.  0.  1.4]
Molecule Charge: 0
Number of Electrons: 4
Number of Qubits: 8
Number of Pauli Strings: 185
Occupied Index: [0 1 2 3]
Number of Single Excitations: 16
Number of Double Excitations: 36
Total Number of Excitations: 52
Hartree-Fock State: [1 1 1 1 0 0 0 0]

Disentangle UCCSD Excitation Order:
[[3, 7], [3, 6], [3, 5], [3, 4], [0, 3, 4, 5], [1, 3, 4, 5], [2, 3, 4, 5], [0, 3, 4, 6], [1, 3, 4, 6], [2, 3, 4, 6], [0, 3, 4, 7], [1, 3, 4, 7], [2, 3, 4, 7], [0, 3, 5, 6], [1, 3, 5, 6], [2, 3, 5, 6], [0, 3, 5, 7], [1, 3, 5, 7], [2, 3, 5, 7], [0, 3, 6, 7], [1, 3, 6, 7], [2, 3, 6, 7], [2, 7], [2, 6], [2, 5], [2, 4], [0, 2, 4, 5], [1, 2, 4, 5], [0, 2, 4, 6], [1, 2, 4, 6], [0, 2, 4, 7], [1, 2, 4, 7], [0, 2, 5, 6], [1, 2, 5, 6], [0, 2, 5, 7], [1, 2, 5, 7], [0, 2, 6, 7], [1, 2, 6, 7], [1, 7], [1, 6], [1, 5], [1, 4], [0, 1, 4, 5], [0, 1, 4, 6], [0, 1, 4, 7], [0, 1, 5, 6], [0, 1, 5, 7], [0, 1, 6, 7], [0, 7], [0, 6], [0, 5], [0, 4]]

 0: ──╭

### Info: Lithium Hydride (STO-3G)

In [13]:
# Create Molecule
mol_name = 'lih'
mol_full_name = 'lithium_hydride'
bond_length = 1.4
mol_symbols, coords, charge, num_elec = getmol.get_molecule(mol_name, bond_length)
print(f'Molecule Symbol: {mol_symbols}')
print(f'Coordinates: {coords}')
print(f'Molecule Charge: {charge}')
print(f"Number of Electrons: {num_elec}")

# Get the Molecular Hamiltonian (Using Jordan-Wigner)
mol_ham, num_qubits = qml.qchem.molecular_hamiltonian(symbols=mol_symbols, coordinates=coords, name=mol_full_name ,charge=charge, basis='sto-3g')
# print(f'Molecular Hamiltonian:\n{mol_ham}')
print(f'Number of Qubits: {num_qubits}')
print(f"Number of Pauli Strings: {len(mol_ham.terms[1])}")

# Getting the Orbital and Excitation Index 
occupied_index, single_index, double_index = getind.generate_excite_index(num_qubits, num_elec)
print(f"Occupied Index: {occupied_index}")
# print(f"Single Excitation Index: \n{single_index}")
# print(f"Double Excitation  Index:\n{double_index}")

# Get the Fermionic Disentangled Unitary Coupled Cluster Single Doubles (UCCSD) Excitation Order 
disentangle_order = getind.generate_disentanglement_order(occupied_index, single_index, double_index)
# print(f"Disentangle UCCSD Excitation Order:\n{disentangle_order}")
print(f"Number of Single Excitations: {len(single_index)}")
print(f"Number of Double Excitations: {len(double_index)}")
print(f"Total Number of Excitations: {len(disentangle_order)}")

# Get the Hartree-Fock State
hf = qml.qchem.hf_state(electrons=num_elec, orbitals=num_qubits)
print(f"Hartree-Fock State: {hf}\n")

# Prepare the Device
dev = qml.device('default.qubit', wires=num_qubits)

@qml.qnode(dev) # Prepare the UCCSD Ansatz based on the Disentangled Order
def uccsd_ansatz(params, wires, excitations):
    qml.BasisState(hf, wires=wires)
    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        else:
            qml.SingleExcitation(params[i], wires=excitation)

    return qml.expval(qml.PauliZ(0))

print(f"Disentangle UCCSD Excitation Order:\n{disentangle_order}\n") 

# Draw the quantum circuit
drawer = qml.draw(uccsd_ansatz, max_length=100)
print(drawer(params=plnp.zeros(len(disentangle_order)), wires=range(num_qubits), excitations=disentangle_order))

Molecule Symbol: ['Li', 'H']
Coordinates: [0.  0.  0.  0.  0.  1.4]
Molecule Charge: 0
Number of Electrons: 4
Number of Qubits: 12
Number of Pauli Strings: 631
Occupied Index: [0 1 2 3]
Number of Single Excitations: 32
Number of Double Excitations: 168
Total Number of Excitations: 200
Hartree-Fock State: [1 1 1 1 0 0 0 0 0 0 0 0]

Disentangle UCCSD Excitation Order:
[[3, 11], [3, 10], [3, 9], [3, 8], [3, 7], [3, 6], [3, 5], [3, 4], [0, 3, 4, 5], [1, 3, 4, 5], [2, 3, 4, 5], [0, 3, 4, 6], [1, 3, 4, 6], [2, 3, 4, 6], [0, 3, 4, 7], [1, 3, 4, 7], [2, 3, 4, 7], [0, 3, 4, 8], [1, 3, 4, 8], [2, 3, 4, 8], [0, 3, 4, 9], [1, 3, 4, 9], [2, 3, 4, 9], [0, 3, 4, 10], [1, 3, 4, 10], [2, 3, 4, 10], [0, 3, 4, 11], [1, 3, 4, 11], [2, 3, 4, 11], [0, 3, 5, 6], [1, 3, 5, 6], [2, 3, 5, 6], [0, 3, 5, 7], [1, 3, 5, 7], [2, 3, 5, 7], [0, 3, 5, 8], [1, 3, 5, 8], [2, 3, 5, 8], [0, 3, 5, 9], [1, 3, 5, 9], [2, 3, 5, 9], [0, 3, 5, 10], [1, 3, 5, 10], [2, 3, 5, 10], [0, 3, 5, 11], [1, 3, 5, 11], [2, 3, 5, 11], [0, 3,